# Word2Vec 

## 主要思想：

**一个词的含义应该是由该词的上下文来决定的。**

1、我们有一个很大的语料库，这个语料库包含了很多的 text

2、所有的单词都可以用一个向量表示

3、我们使用一个中心词 $c$ 和上下文 $o$ 来遍历 text 中的每个位置 $t$

4、使用 $c$ 和 $o$ 的词向量的相似度来计算 $P(o|c)$

5、最大化这个概率。

6、上述步骤结束之后，我们得到的权重矩阵，就可以看作是单词的 vector


### 两个模型：

* skip-grams: 给定单词来预测上下文

* CBOW: 给定上下文来预测单词

### 两个高效的训练方法：

* Hierarchical Softmax (这东西我看的也不是很懂，秉着不求甚解的精神，我决定不看这个了)

* Negative sampling


这里的模型 + 训练方法一共可以有四种组合。

这里，只会写 skip-grams 模型以及 skip-grams + Negative sampling，其余内容如果有兴趣，可以自行编写，或者以后我想写了再补齐（就是这么任性~）

这里只简单地讲述主要思想和实现方式，更多的内容，可以观看 [CS224N 第二课](https://www.bilibili.com/video/av13383754/?from=search&seid=17986192997424429774#page=2) 和 [CS224N 第三课](https://www.bilibili.com/video/av13383754/?from=search&seid=17986192997424429774#page=3)


## skip-grams模型


奉献一张，我心中认为的解读 skip-grams模型 最棒的图，一图胜千言。

![skip-grams](./word2vec/skipgram.jpg)

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# 构建 skip-grams 模型

class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        """
            vocab_size: 语料库中的单词的数量
            emb_dim: 向量的维度
        """
        super(SkipGramModel, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.embedding_v = nn.Embedding(vocab_size, emb_dim)
        self.embedding_u = nn.Embedding(vocab_size, emb_dim)
        self.init_emb()
    
    def forward(center_words, target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D
        
        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1 
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV
        
        P_oc = torch.exp(scores) / torch.sum(torch.exp(norm_scores), 1).unsqueeze(1) # Bx1 * Bx1 => B*1 表示 B个单词的 P(o|c)
        nll = -torch.mean(torch.log(P_oc)) # 求 这 B 个单词的 negative log likelihood

        return nll 
    
    def init_emb(self):
        """
            初始化网络权重
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        return 
    
    def prediction(self, inputs):
        """
            给出 word 的 vector, 这里，我们没有用到后面的 embedding_u，其实这里还有几种做法：
            1、embedding_v + embedding_u； 
            2、embedding_v[:, N/2; :] + embedding_u[:; :, N/2] 也就是 embedding_v 取前一半， embedding_u取后一半，两者拼接。
        """
        embeds = self.embedding_v(inputs)
        return embeds 
    

Todo: 找一个小词库，自己过一遍训练词向量的流程。

## skip-grams模型 +  Negative sampling

为什么需要使用 Negative sampling ？？

